In [7]:
import csv
import time
from ytmusicapi import YTMusic

yt = YTMusic()

In [8]:
def expand_network(input_csv="mcr_similarity_network.csv"):
    new_relations = []
    processed_artists = set()

    # 1. Baca data dari Level 1
    with open(input_csv, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        seeds = [row['target'] for row in reader]
        # Tambahkan MCR sendiri ke daftar yang sudah diproses agar tidak duplikat
        processed_artists.add("My Chemical Romance")

    print(f"Memulai ekspansi untuk {len(seeds)} artis...")

    for artist_name in seeds:
        if artist_name in processed_artists: continue
        
        print(f"Mencari kemiripan untuk: {artist_name}...")
        try:
            search = yt.search(artist_name, filter="artists")
            if not search: continue
            
            artist_id = search[0]['browseId']
            artist_data = yt.get_artist(artist_id)
            related = artist_data.get('related', {}).get('results', [])

            for r in related[:5]: # Ambil 5 saja agar tidak terlalu banyak/kena banned
                new_relations.append({
                    'source': artist_name,
                    'target': r['title'],
                    'target_id': r['browseId'],
                    'relationship': 'Similar',
                })
            
            processed_artists.add(artist_name)
            time.sleep(1) # Jeda aman agar tidak dianggap bot
        except Exception as e:
            print(f"Gagal mengambil data {artist_name}: {e}")

    # 2. Simpan hasil ekspansi ke file baru
    with open("expanded_network.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['source', 'target', 'target_id', 'relationship'])
        writer.writeheader()
        writer.writerows(new_relations)
    
    print("Ekspansi Selesai! File 'expanded_network.csv' siap.")

expand_network()


Memulai ekspansi untuk 10 artis...
Mencari kemiripan untuk: Panic! At The Disco...
Mencari kemiripan untuk: Bring Me the Horizon...
Mencari kemiripan untuk: Green Day...
Mencari kemiripan untuk: Avenged Sevenfold...
Mencari kemiripan untuk: The Used...
Mencari kemiripan untuk: Fall Out Boy...
Mencari kemiripan untuk: Neck Deep...
Mencari kemiripan untuk: Pierce The Veil...
Mencari kemiripan untuk: Paramore...
Mencari kemiripan untuk: Alesana...
Ekspansi Selesai! File 'expanded_network.csv' siap.
